# 11. Simple CNN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.init
from torch.autograd import Variable

import torchvision.utils as utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.utils.data as Data

import numpy as np
import random
import os

## 11.1 Preparing Custom Data

In [2]:
img_dir = "./image"
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
            transforms.Grayscale(),
            transforms.ToTensor(),
            ]))

#https://pytorch.org/docs/stable/torchvision/transforms.html

print(img_data.classes)
print(img_data.class_to_idx)

['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
{'ㄱ': 0, 'ㄲ': 1, 'ㄴ': 2, 'ㄷ': 3, 'ㄸ': 4, 'ㄹ': 5, 'ㅁ': 6, 'ㅂ': 7, 'ㅃ': 8, 'ㅅ': 9, 'ㅆ': 10, 'ㅇ': 11, 'ㅈ': 12, 'ㅉ': 13, 'ㅊ': 14, 'ㅋ': 15, 'ㅌ': 16, 'ㅍ': 17, 'ㅎ': 18, 'ㅏ': 19, 'ㅐ': 20, 'ㅑ': 21, 'ㅒ': 22, 'ㅓ': 23, 'ㅔ': 24, 'ㅕ': 25, 'ㅖ': 26, 'ㅗ': 27, 'ㅘ': 28, 'ㅙ': 29, 'ㅛ': 30, 'ㅜ': 31, 'ㅝ': 32, 'ㅞ': 33, 'ㅟ': 34, 'ㅠ': 35, 'ㅡ': 36, 'ㅢ': 37, 'ㅣ': 38}


In [3]:
batch_size = 100
font_num = 720

In [4]:
def train_test_split(data, train_ratio, stratify, stratify_num, batch_size) :
    
    length = len(data)
    
    if stratify :
        label_num = int(len(data)/stratify_num)
        cut = int(stratify_num*train_ratio)
        train_indices = np.random.permutation(np.arange(stratify_num))[:cut]
        test_indices = np.random.permutation(np.arange(stratify_num))[cut:]
        
        for i in range(1, label_num) :
            train_indices = np.concatenate((train_indices, np.random.permutation(np.arange(stratify_num))[:cut] + stratify_num*i))
            test_indices = np.concatenate((test_indices, np.random.permutation(np.arange(stratify_num))[cut:] + stratify_num*i))
        
    else :
        cut = int(len(data)*train_ratio)
        train_indices = np.random.permutation(np.arange(length))[:cut]
        test_indices = np.random.permutation(np.arange(length))[cut:]
        
    train_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = train_indices, num_workers=1, drop_last = True)
    test_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = test_indices, num_workers=1, drop_last = True)

    return train_loader, test_loader, len(train_indices), len(test_indices)

In [5]:
train_loader, test_loader, train_num, test_num = train_test_split(img_data, 0.8, True, font_num, batch_size)

## 11.2 Define Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*5*5,100),
            nn.ReLU(),
            nn.Linear(100,39)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

In [7]:
model = CNN().cuda()

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [9]:
num_epochs = 5

In [10]:
for epoch in range(num_epochs):

    for i, (batch_images, batch_labels) in enumerate(train_loader):

        X = Variable(batch_images).cuda()
        Y = Variable(batch_labels).cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, train_num//batch_size, cost.data[0]))

Epoch [1/5], lter [100/224] Loss: 3.2402
Epoch [1/5], lter [200/224] Loss: 2.7495
Epoch [2/5], lter [100/224] Loss: 3.2947
Epoch [2/5], lter [200/224] Loss: 3.0117
Epoch [3/5], lter [100/224] Loss: 2.1730
Epoch [3/5], lter [200/224] Loss: 1.0795
Epoch [4/5], lter [100/224] Loss: 3.4386
Epoch [4/5], lter [200/224] Loss: 2.8769
Epoch [5/5], lter [100/224] Loss: 3.8860
Epoch [5/5], lter [200/224] Loss: 3.5920


## 11.3 Test Model

In [11]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = Variable(images).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test images: %f %%' % (100 * correct / total))

Accuracy of test images: 2.571429 %
